<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_AWS_MARCH2024_P0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colab-env --quiet

!pip install sagemaker boto3 --quiet

#%pip install langchain==0.0.309 --quiet --root-user-action=ignore
%pip install langchain --quiet

import colab_env
import os

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 11.3 MB

In [2]:
aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("AWS_DEFAULT_REGION")
output=os.getenv("AWS_DEFAULT_OUTPUT")

#print(aws_access_key_id)
#print()
#print(f"aws_access_key_id: '{aws_access_key_id}'")
#print(f"aws_secret_access_key: '{aws_secret_access_key}'")

#print(f"region: '{region}'")
#print()

llama-2
https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/

claude-3
https://aws.amazon.com/blogs/aws/anthropics-claude-3-sonnet-foundation-model-is-now-available-in-amazon-bedrock/


In [3]:
import colab_env
import boto3
import os
import sagemaker
from sagemaker.jumpstart.model import JumpStartModel

# added by frank morales december 13, 2023
iam_client = boto3.client("iam")

role = iam_client.get_role(
    RoleName=os.getenv("ROLENAME")
)

ROLE_ARN = role['Role']['Arn']

# https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-choose.html#jumpstart-foundation-models-choose-eula

#original
#llm_model_id, llm_model_version = "meta-textgeneration-llama-2-7b-f", "*"
#llm_model = JumpStartModel(model_id=llm_model_id, model_version=llm_model_version)
## error below

#modified by frankmorales
#llm_model_id, llm_model_version = "meta-textgeneration-llama-2-7b-f", "2.*"

### LLAMA-2
#llm_model_id = 'meta-textgeneration-llama-2-7b-f'
#llm_model_version = '2.0.4'

# https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html

# https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/examples?provider=Anthropic

## MISTRAL
llm_model_id = 'huggingface-llm-mistral-7b'
llm_model_version = '2.1.0'

llm_model = JumpStartModel(model_id=llm_model_id, model_version=llm_model_version, role=ROLE_ARN, region='us-east-1')


#{
#  "modelId": "anthropic.claude-v2",
#  "contentType": "application/json",
#  "accept": "application/json",


#modified by frankmorales
llm_predictor = llm_model.deploy(accept_eula=True)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
--------!

In [4]:
#this is the model endpoint NAME, not the ARN
llm_model_endpoint_name = llm_predictor.endpoint_name
llm_model_endpoint_name

'hf-llm-mistral-7b-2024-03-05-18-43-18-641'

In [5]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
import json

class QAContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps(
            {"inputs" : [
                [
                    {
                        "role" : "system",
                        "content" : ""
                    },
                    {
                        "role" : "user",
                        "content" : prompt
                    }
                ]],
                "parameters" : {**model_kwargs}
            })
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        ### LLAMA-2
        #return response_json[0]["generation"]["content"]
        ### MISTRAL
        return response_json[0]['generated_text']["content"]

qa_content_handler = QAContentHandler()



In [6]:
class QAContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

qa_content_handler = QAContentHandler()

In [7]:
aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("region")
output=os.getenv("output")

llm = SagemakerEndpoint(
        endpoint_name=llm_model_endpoint_name,
        region_name=region,
        model_kwargs={"max_new_tokens": 5000, "top_p": 0.9, "temperature": 1e-11},
        endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
        content_handler=qa_content_handler
    )

In [8]:
### LLAMA-2 ####

#original by the book
#query = "WHAT IS PERU?"

##modified by Frank Morales
#response=llm.predict(query)

In [9]:
#### MISTRAL ######

#original by the book
query = "how AWS evolved?"


# Create a boto3 client for SageMaker runtime
sm_client = boto3.client('runtime.sagemaker')

# Prepare the input for the model
#input_data = {"inputs": query}

### WITH PARAMETRS
n=5
MNT=512*n
model_kwargs={"max_new_tokens": MNT, "temperature": 0.9}
input_data = ({"inputs": query, "parameters" : {**model_kwargs}})

response = sm_client.invoke_endpoint(EndpointName=llm_model_endpoint_name, Body=json.dumps(input_data), ContentType="application/json")

# Decode the response from the model
response_body = json.loads(response['Body'].read().decode('utf-8'))
#print(response_body)

print(f'Query:', query)
print()
print(f'Response:', response_body[0]['generated_text'])

Query: how AWS evolved?

Response: 

Anurag: AWS is a collection of IaaS and PaaS services put together in a cloud platform. Almost all companies that are either building new applications, or are looking to scale their existing applications, to leverage cloud computing. AWS is one of the leaders in the market place, providing public cloud.  Also, it’s not just for building applications – so it’s also the choice of companies that are moving from their existing data centers to build a new data center capability in AWS.

We have gone from Business Intelligence tools in 2004-05, to the whole area of Data Warehousing, which is quite mature; the next step for many businesses is cloud computing. This year, if you go into the major vendors’ websites, many big business applications like E-mail and CRM, will be available as Software as a Service (SaaS) platforms. So, the whole scope for the next few years is moving this application, that runs inside the companies data center to a SaaS applicatio

# CLEAN UP

In [10]:
#  Frank Morales created this cell on December 14, 2023; it fully allows automatically the deletion of endpoints, models, and endpoint configurations.

import colab_env
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region=os.getenv("AWS_DEFAULT_REGION")
aws_output=os.getenv("AWS_DEFAULT_OUTPUT")

import boto3

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

def cleanup_sagemaker_resources(resource_name,resourceid):

    if resourceid==0:
       response=sagemaker_client.list_endpoints()
    elif resourceid==1:
         response=sagemaker_client.list_models()
    elif resourceid==2:
         response=sagemaker_client.list_endpoint_configs()

    print(resource_name)

    number_of_endpoints=len(response['%s'%resource_name])
    for i in range(number_of_endpoints):
        resource_nametmp='%s'%resource_name[0:len(resource_name)-1]
        print('%sName'%resource_nametmp)
        print(response['%s'%resource_name][i]['%sName'%resource_nametmp])

        if resourceid==0:
           endpoint_name=response['%s'%resource_name][i]['%sName'%resource_nametmp]
           sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        elif resourceid==1:
           sagemaker_client.delete_model(ModelName=response['Models'][i]['ModelName'])
        elif resourceid==2:
           sagemaker_client.delete_endpoint_config(EndpointConfigName=response['EndpointConfigs'][i]['EndpointConfigName'])

    print("\n==================================\n")


cleanup_sagemaker_resources('Endpoints',0)
cleanup_sagemaker_resources('Models',1)
cleanup_sagemaker_resources('EndpointConfigs',2)

Endpoints
EndpointName
hf-llm-mistral-7b-2024-03-05-18-43-18-641


Models
ModelName
hf-llm-mistral-7b-2024-03-05-18-43-18-639


EndpointConfigs
EndpointConfigName
hf-llm-mistral-7b-2024-03-05-18-43-18-641


